# Dataset loading script for HotpotQA QG

In [1]:
from datasets import load_dataset 

hotpotqa_distractor = load_dataset("hotpot_qa","distractor")

/home/deokhk/miniconda3/envs/DIRE/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset hotpot_qa (/home/deokhk/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)
100%|██████████| 2/2 [00:00<00:00, 420.90it/s]


In [4]:
single_datapoint = hotpotqa_distractor["train"][0]

question = single_datapoint["question"]
answer = single_datapoint["answer"]
supporting_facts = single_datapoint["supporting_facts"]
context = single_datapoint["context"]

In [5]:
title_to_contextidx = dict()
for idx, context_title in enumerate(context["title"]):
    title_to_contextidx[context_title] = idx

In [6]:
qg_context = ""
sp_titles = supporting_facts["title"]
sp_sent_ids = supporting_facts["sent_id"]

for sp_title, sp_sent_id in zip(sp_titles, sp_sent_ids):
    contextidx = title_to_contextidx[sp_title]
    corresponding_sp = context["sentences"][contextidx][sp_sent_id]
    qg_context += (corresponding_sp +" ")


In [7]:
# highlight answer 
answer_start_idx = qg_context.find(answer)
answer_end_idx = answer_start_idx + len(answer)

que_gen_input = f"generate question: {qg_context[:answer_start_idx]} <hl> {answer} <hl> {qg_context[answer_end_idx:]}"


In [12]:
que_gen_input

"generate question:  <hl> Arthur's Magazine <hl>  (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. First for Women is a woman's magazine published by Bauer Media Group in the USA. "

In [14]:
que_gen_target = question 

# Input processing for HotpotQA-based longformer model

In [8]:
from datasets import load_dataset 

hotpotqa_distractor = load_dataset("hotpot_qa","distractor")
hotpotqa_train_datapoint = hotpotqa_distractor["train"][0]

Found cached dataset hotpot_qa (/home/deokhk/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)
100%|██████████| 2/2 [00:00<00:00, 703.39it/s]


In [9]:
hotpotqa_train_datapoint

{'id': '5a7a06935542990198eaf050',
 'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'answer': "Arthur's Magazine",
 'type': 'comparison',
 'level': 'medium',
 'supporting_facts': {'title': ["Arthur's Magazine", 'First for Women'],
  'sent_id': [0, 0]},
 'context': {'title': ['Radio City (Indian radio station)',
   'History of Albanian football',
   'Echosmith',
   "Women's colleges in the Southern United States",
   'First Arthur County Courthouse and Jail',
   "Arthur's Magazine",
   '2014–15 Ukrainian Hockey Championship',
   'First for Women',
   'Freeway Complex Fire',
   'William Rast'],
  'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.",
    ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
    ' It plays Hindi, English and regional songs.',
    ' It was launch

# hotpotqa_qa_dataset script
# 애초에 probing dataset은 context 자체에서 빼야되는 SP가 포함된 paragraph는 제외되어있구나 


In [10]:
import torch
import datasets
from transformers import LongformerTokenizerFast

tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')


In [11]:
hotpotqa_distractor = load_dataset("hotpot_qa","distractor")
datapoint = hotpotqa_distractor["train"][0]

Found cached dataset hotpot_qa (/home/deokhk/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)
100%|██████████| 2/2 [00:00<00:00, 440.60it/s]


In [12]:
hotpotqa_distractor

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})

In [13]:
datapoint

{'id': '5a7a06935542990198eaf050',
 'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'answer': "Arthur's Magazine",
 'type': 'comparison',
 'level': 'medium',
 'supporting_facts': {'title': ["Arthur's Magazine", 'First for Women'],
  'sent_id': [0, 0]},
 'context': {'title': ['Radio City (Indian radio station)',
   'History of Albanian football',
   'Echosmith',
   "Women's colleges in the Southern United States",
   'First Arthur County Courthouse and Jail',
   "Arthur's Magazine",
   '2014–15 Ukrainian Hockey Championship',
   'First for Women',
   'Freeway Complex Fire',
   'William Rast'],
  'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.",
    ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
    ' It plays Hindi, English and regional songs.',
    ' It was launch

In [14]:
mycontext = ""
context_titles = datapoint["context"]["title"]
context_sentences_list = datapoint["context"]["sentences"]
question = datapoint["question"]

In [15]:
mycontext = f"[q] {question} [/q] "
for title, sentence_list in zip(context_titles, context_sentences_list):
    mycontext += (f"<t> {title} </t> ")
    for sentence in sentence_list:
        mycontext += (f"{sentence} [s] ")

In [16]:
# answer span은 
answer = datapoint["answer"]
answer_idx = mycontext.index(answer)

In [17]:
answer_idx

37

In [18]:
from transformers import LongformerForQuestionAnswering, LongformerTokenizerFast, EvalPrediction

tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', additional_special_tokens=['[q]', '[/q]', '<t>', '</t>', '[s]'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
tokenizer.additional_special_tokens

['[q]', '[/q]', '<t>', '</t>', '[s]']

In [31]:
encodings = tokenizer(mycontext, max_length=4096, padding="max_length")
# Global attention mask 만드는 녀석 생성. <s>, </s> 


In [33]:
tokens = tokenizer.decode(encodings["input_ids"])

In [37]:
special_token_ids_list = tokenizer('[q][/q]<t></t>[s]<s></s>')["input_ids"]

In [38]:
special_token_ids_list = list(set(special_token_ids_list))

[0, 50265, 50266, 50267, 50268, 50269, 0, 2, 2]

In [39]:
import numpy as np 

def func_global_att(x):
    if x in special_token_ids_list:
        return 1 
    else:
        return 0


input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]
global_attention_mask = [0 for x in input_ids if x in special_token_ids_list]

In [34]:
tokens

'<s>[q] Which magazine was started first Arthur\'s Magazine or First for Women? [/q] <t> Radio City (Indian radio station) </t> Radio City is India\'s first private FM radio station and was started on 3 July 2001. [s]  It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003). [s]  It plays Hindi, English and regional songs. [s]  It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007. [s]  Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features. [s]  The Radio station currently plays a mix of Hindi and Regional music. [s]  Abraham Thomas is the CEO of the company. [s] <t> History of Albanian football </t> Football in Albania existed before the Albanian Football Federation (FSHF) was create

In [32]:
encodings

{'input_ids': [0, 50265, 6834, 4320, 21, 554, 78, 9411, 18, 10202, 50, 1234, 13, 2691, 116, 1437, 50266, 1437, 50267, 4611, 412, 36, 25767, 3188, 1992, 43, 1437, 50268, 4611, 412, 16, 666, 18, 78, 940, 12895, 3188, 1992, 8, 21, 554, 15, 155, 550, 5155, 4, 1437, 50269, 1437, 85, 24798, 15, 8301, 4, 134, 36, 4352, 14730, 8301, 4, 288, 11, 144, 1947, 43, 14251, 1843, 12527, 31, 5729, 36, 8569, 24, 21, 554, 11, 4482, 238, 7104, 5515, 36, 33437, 78, 11, 5155, 238, 13389, 8310, 8, 188, 3534, 36, 30117, 4999, 322, 1437, 50269, 1437, 85, 1974, 19840, 6, 2370, 8, 2174, 3686, 4, 1437, 50269, 1437, 85, 21, 1660, 11, 17502, 7826, 11, 494, 3503, 6, 11, 17494, 15, 262, 550, 3503, 8, 11, 10035, 7352, 1115, 415, 8697, 779, 3010, 4, 1437, 50269, 1437, 4611, 412, 682, 13, 12198, 88, 188, 2454, 11, 392, 2266, 19, 5, 1709, 9, 10, 930, 16698, 111, 14839, 500, 5416, 36139, 4, 175, 14, 1523, 930, 1330, 340, 6, 3424, 6, 3686, 6, 8, 97, 930, 12, 3368, 1575, 4, 1437, 50269, 1437, 20, 4611, 1992, 855, 1974, 10, 

In [76]:
input_ids = encodings["input_ids"]
out = tokenizer.convert_ids_to_tokens(input_ids)

In [77]:
out2 = tokenizer.convert_tokens_to_string(out)

In [62]:
out2

'<s>[q] Which magazine was started first Arthur\'s Magazine or First for Women? [/q] <t> Radio City (Indian radio station) </t> Radio City is India\'s first private FM radio station and was started on 3 July 2001. [s]  It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003). [s]  It plays Hindi, English and regional songs. [s]  It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007. [s]  Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features. [s]  The Radio station currently plays a mix of Hindi and Regional music. [s]  Abraham Thomas is the CEO of the company. [s] <t> History of Albanian football </t> Football in Albania existed before the Albanian Football Federation (FSHF) was create